# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Useful links:

- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:

- 40 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set
- 20 points - Evaluate on Github Typo Corpus (for masters only)

Remarks:

- Use Python 3 or greater
- Max is 80 points for bachelors, 100 points for masters


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html).

[N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf)

You may also wnat to implement:

- spell-checking for a concrete language - Russian, Tatar, Ukranian, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

Important - your project should not be a mere code copy-paste from somewhere. Implement yourself, analyze why it was suggested this way, and think of improvements/customization.

Your solution should be able to perform 4 corrections per second (3-5 words in an example) on a typical cpu.


## Justify your decisions

In your implementation you will need to decide which ngram dataset to use, which weights to assign for edit1, edit2 or absent words probabilities, beam search parameters and etc. Write down justificaitons for these choices.


## My solution


In [232]:
from collections import Counter


class MySolution:
    """
        I will implement all the functions needed for my solution in this class.
    """

    def __init__(self, ngram_file):
        self.ngram_file = ngram_file
        self.calc_words()
        self.conditional_probs = self.calc_conditional_probs()
        self.conditional_probs_weight = 0.9352175845886633
        self.frequency_weight = 0.03328620660509829
        self.edit_distance_weight = 0.8554663627569048

    def calc_conditional_probs(self):
        '''
            To calculate the conditional probability from the ngram model
        '''
        conditional_probs = {}
        for two_words, count_w1_w2 in self.w1_w2_counter.items():
            w1, _ = two_words.split('_')
            conditional_probs[two_words] = int(
                count_w1_w2) / self.w1_counter[w1]
        return conditional_probs

    def calc_words(self):
        '''
            Collecting the information needed to build the ngram model
        '''
        self.words = []
        self.w1_w2_counter = {}
        self.w1_counter = {}
        with open(self.ngram_file, 'r', encoding='latin-1') as f:
            for line in f.readlines():
                count, w1, w2 = line.split()
                self.w1_w2_counter[w1+'_'+w2] = count
                if self.w1_counter.get(w1):
                    self.w1_counter[w1] += int(count)
                else:
                    self.w1_counter[w1] = int(count)
                self.words.append(w1)
                self.words.append(w2)
        self.words = Counter(self.words)

    def edit_distance(self, word1, word2):
        """Calculate the Levenshtein distance between two words."""
        if len(word1) < len(word2):
            return self.edit_distance(word2, word1)

        if not word2:
            return len(word1)

        previous_row = range(len(word2) + 1)

        for i, c1 in enumerate(word1):
            current_row = [i + 1]

            for j, c2 in enumerate(word2):
                insertions = previous_row[j + 1] + 1
                deletions = current_row[j] + 1
                substitutions = previous_row[j] + (c1 != c2)
                current_row.append(min(insertions, deletions, substitutions))

            previous_row = current_row
        if previous_row[-1] != 0:
            return previous_row[-1]
        return 1

    def edits1(self, word):
        "All edits that are one edit away from `word`."
        letters = 'abcdefghijklmnopqrstuvwxyz'
        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes = [L + R[1:] for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
        replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
        inserts = [L + c + R for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)

    def edits2(self, word): return (e2 for e1 in self.edits1(word)
                                    for e2 in self.edits1(e1))

    def known(self, words): return set(w for w in words if w in self.words)

    def candidates(self, word):
        '''
            Getting all the candidate words that can be the correction of word
        '''
        if len(word) == 1:
            return word
        l = []
        l.extend(self.known(self.edits1(word)))
        if len(word) > 4:
            l.extend(self.known(self.edits2(word)))
        l.append(word)
        return l

    def get_best_word(self, original_word, cands, next_word=None, previous_word=None):
        '''
            Choosing the best word from the candidates list
        '''
        best_word = None
        best_score = 0

        if next_word or previous_word:
            for word in cands:
                gram_format = f'{word}_{next_word}' if next_word else f'{previous_word}_{word}'
                if self.conditional_probs.get(gram_format):
                    score = self.frequency_weight * self.words[word]/len(self.words) + \
                        self.edit_distance_weight * (1/self.edit_distance(word, original_word)) + \
                        self.conditional_probs_weight * \
                        self.conditional_probs[gram_format]

                    if word == original_word and self.known([word]):
                        score += 0.1
                    if score > best_score:
                        best_score = score
                        best_word = word

        if best_word == None:
            # choose the most frequent candidate with the least edit distance
            for word in cands:
                if self.words[word] > 0:
                    score = 0.8 * self.words[word]/len(self.words) + \
                        0.2 * 1/(self.edit_distance(word, original_word))
                    if word == original_word and self.known([word]):
                        score += 0.1
                    if score > best_score:
                        best_word = word
                        best_score = score
        if best_word == None:
            return original_word
        return best_word

    def check(self, sentence):
        '''
            This function takes the sentence as an input and return the same sentence corrected
        '''
        result = []
        # correcting the first word
        first_word = sentence[0]
        if len(sentence) > 1:
            if self.words[first_word] <= 50:  # mean that this word need correction
                next_word = sentence[1]
                cands = self.candidates(first_word)
                best_word = self.get_best_word(first_word, cands, next_word)
                result.append(best_word)
            else:
                result.append(first_word)
        else:
            cands = self.candidates(first_word)
            return self.get_best_word(first_word, cands)

        # correcting the rest of the sentence
        for i in range(1, len(sentence)):
            # if the word frequency is less than 50, then it is probably misspelled
            if self.words[sentence[i]] <= 50:
                cands = self.candidates(sentence[i])
                previous_word = result[i-1]
                best_word = self.get_best_word(
                    sentence[i], cands, previous_word=previous_word)
                result.append(best_word)
            else:
                result.append(sentence[i])
        return result


my_solution = MySolution('w2_.txt')


## What are the problems with Norvig's solution?

- His solution is not able to correct unknown words
- His error model is trivial: the smaller the edit distance, the smaller the error.
- His solution can not look at the context of the word


## My proposed solution to this problems

First of all, the dataset which Norvig's solution was trained on is small and it doesn't contain variaty of words, so in my solution the corpus of words that I used is from the ngram data file which contain large amount of words in different shapes. This limits the number of the unknown words that the model faces.

Secondly, his error model was prioritizing the candidates with small edit distance. However, that is not always the case as. So to make the model more flexable, I allowed the candidates to be a list of all 1 and 2 edits words plus the original word. This solution rely on the fact that the model will make the right decision and choose the suitable word for the given context.

Finally, his solution can not look at the context of the word which increase the probability of making errors while choosing from the candidates. To solve this problem, I used bigram model to look at the context of the word and propose the correction with the highest conditional probability, highest frequenct, and lowest edit distance score in a weighted manner.

### Why did I choose bigram model?

- Bigram models are faster and require less memory than 5-gram models because they consider pairs of adjacent words rather than groups of five words at a time.
- In a bigram model, the probability of a word depends only on the preceding word, so it can handle unseen words better than a 5-gram model, which would require seeing the preceding four words to make a prediction.
- Simplicity and ease of implementation

### Why did I choose these specific weights for the scores of the candidates?

- my_solution.conditional_probs_weight = 0.9352175845886633
- my_solution.frequency_weight = 0.03328620660509829
- my_solution.edit_distance_weight = 0.8554663627569048

Using trial and error on the training dataset, I came to a conclusion that these are the most suitable weights to use.

I assigned these values randomly for 50 trials and took the best preforming weights on the training set and then reported their results on the validation set


## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.


# Creating my own dataset


In [168]:
import re


def words(text): return re.findall(r'\w+', text.lower())


y = []
with open(r'datasets\articles.txt', 'r') as f:  # a collection of articles
    y = [words(line.strip('\n')) for line in f.readlines()]

training_y = y[:int(len(y)*0.8)]
with open(r'datasets\training_y.txt', 'a') as f:
    for line in training_y:
        f.write(' '.join(line) + '\n')

validation_y = y[int(len(y)*0.8):]
with open(r'datasets\validation_y.txt', 'a') as f:
    for line in validation_y:
        f.write(' '.join(line) + '\n')


In [171]:
import random


def misspell_word(word, noise_prob):
    """Misspells a given word with a given noise probability."""
    if random.uniform(0, 1) <= noise_prob:
        # Choose a random letter to replace
        index = random.randint(0, len(word) - 1)
        new_char = chr(random.randint(ord('a'), ord('z')))
        return word[:index] + new_char + word[index + 1:]
    else:
        return word


def misspell_text(text, noise_prob):
    """Misspells all the words in a given text with a given noise probability."""
    result = []
    for sentence in text:
        misspelled_sentece = [misspell_word(
            word, noise_prob) for word in sentence.split]
        result.append(misspelled_sentece)
    return result


# Generate a misspelled version of the text with a given noise probability
noise_prob = 0.5
training_x = misspell_text(training_y, noise_prob)
validation_x = misspell_text(validation_y, noise_prob)


In [178]:
def evaluate(model, x, y):
    total_words = 0
    my_correct = 0
    for i, sentence in enumerate(x):
        if len(sentence):
            result = model.check(sentence)
            for j in range(len(sentence)):
                if result[j] == y[i][j]:
                    my_correct += 1
                total_words += 1
    return my_correct/total_words


Here I carry out the trails that I mentioned earlier to find which weights are the most suitable


In [195]:
import random
max_accuracy = 0
weights = ()
for i in range(50):
    print(f'Trial {i+1}')
    my_solution = MySolution('w2_.txt')
    my_solution.conditional_probs_weight = random.random()
    my_solution.frequency_weight = random.random()
    my_solution.edit_distance_weight = random.random()
    acc = evaluate(my_solution, training_x[:100], training_y[:100])
    if acc > max_accuracy:
        max_accuracy = acc
        weights = (my_solution.conditional_probs_weight,
                   my_solution.frequency_weight, my_solution.edit_distance_weight)
    print(acc)
    print((my_solution.conditional_probs_weight,
          my_solution.frequency_weight, my_solution.edit_distance_weight))
print(max_accuracy)
print(weights)


Trial 1
0.8316008316008316
(0.8866385785933448, 0.5426769478432463, 0.2997161569521408)
Trial 2
0.8316008316008316
(0.9514580737088721, 0.9175060857503533, 0.8612165274009033)
Trial 3
0.8295218295218295
(0.1851382537582692, 0.2996027160810769, 0.28515713507481155)
Trial 4
0.8316008316008316
(0.9534656165705472, 0.9675053070791463, 0.5581648301554646)
Trial 5
0.8295218295218295
(0.17889009631087016, 0.4603203375140825, 0.14740059281681372)
Trial 6
0.8357588357588358
(0.42335956436369326, 0.2490727496466537, 0.8449369489075931)
Trial 7
0.8316008316008316
(0.5977778294057307, 0.7076233553841301, 0.2919550152153986)
Trial 8
0.8305613305613305
(0.09462556453806403, 0.4544999058014326, 0.9254127649797096)
Trial 9
0.8357588357588358
(0.4705326604848764, 0.1822928371615714, 0.8475318967377594)
Trial 10
0.8232848232848233
(0.21740419746182948, 0.5409155846453342, 0.024890286065343936)
Trial 11
0.8295218295218295
(0.5518473477426818, 0.8663907895925044, 0.6850918714303492)
Trial 12
0.83679833679

In [196]:
print(max_accuracy)
print(weights)


0.840956340956341
(0.9352175845886633, 0.03328620660509829, 0.8554663627569048)


## Norvig's solution


In [244]:
import re
from collections import Counter


def words(text): return re.findall(r'\w+', text.lower())


WORDS = Counter(words(open('big.txt').read()))


def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    return WORDS[word] / N


def correction(sentence):
    "Most probable spelling correction for word."
    result = []
    for word in sentence:
        result.append(max(candidates(word), key=P))
    return result


def candidates(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])


def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)


def edits1(word):
    "All edits that are one edit away from `word`."
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)


def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))


correction(['value'])


['value']

## Testing both solutions


In [238]:
my_solution = MySolution('w2_.txt')


def correction(word):
    return my_solution.check([word])


def unit_tests():
    assert correction('speling') == 'spelling'              # insert
    assert correction('korrectud') == 'corrected'           # replace 2
    assert correction('bycycle') == 'bicycle'               # replace
    assert correction('inconvient') == 'inconvenient'       # insert 2
    assert correction('arrainged') == 'arranged'            # delete
    # transpose + delete
    assert correction('peotryy') == 'poetry'
    assert correction('word') == 'word'                     # known
    assert correction('quintessential') == 'quintessential'  # unknown
    assert words('This is a TEST.') == ['this', 'is', 'a', 'test']
    assert Counter(words('This is a test. 123; A TEST this is.')) == (
        Counter({'123': 1, 'a': 2, 'is': 2, 'test': 2, 'this': 2}))
    return 'unit_tests pass'


unit_tests()


'unit_tests pass'

I wrote a paragraph about philosophy and added some misspelled words to it


In [237]:
# loading the dataset
import re
import time


def words(text): return re.findall(r'\w+', text.lower())


misspelled = 'p1_misspelled.txt'
correct = 'p1.txt'
x = []
with open(misspelled, 'r') as f:
    x = [words(line.strip('\n')) for line in f.readlines()]

y = []
with open(correct, 'r') as f:
    y = [words(line.strip('\n')) for line in f.readlines()]


my_solution = MySolution('w2_.txt')
my_solution.conditional_probs_weight = 0.9352175845886633
my_solution.frequency_weight = 0.03328620660509829
my_solution.edit_distance_weight = 0.8554663627569048

total_words = 0
my_correct = 0
norv_correct = 0
wrong = []
wrong2 = []
start = time.time()
for i, sentence in enumerate(x):
    my_result = my_solution.check(sentence)
    norvig_result = correction(sentence)

    for j in range(len(sentence)):
        if my_result[j] == y[i][j]:
            my_correct += 1
        else:
            wrong.append((my_result[j], y[i][j], x[i][j], my_result[j-1]))
        if norvig_result[j] == y[i][j]:
            norv_correct += 1
        else:
            wrong2.append(norvig_result[j])
        total_words += 1
dt = time.time() - start
print(
    f'My solution got {my_correct/total_words*100}% at {total_words/dt} words per second')
print(f'Norvig solution got {norv_correct/total_words*100}%')


My solution got 97.43589743589743% at 54.956868808326085 words per second
Norvig solution got 94.01709401709401%


# Conclusion


As you can see, my solution preformed better than Norvig solution. For farther work, I suggest using linear regression to assign the weights of the frequency, edit distance, and conditional probability.


## Evaluate on Github Typo Corpus

Now, you need to evaluate your solution on the Github Typo Corpus. Don't forget to compare the accuracy with the Norvig's solution.


In [1]:
!wget https: // github-typo-corpus.s3.amazonaws.com/data/github-typo-corpus.v1.0.0.jsonl.gz
!gzip - d github-typo-corpus.v1.0.0.jsonl.gz


'wget' is not recognized as an internal or external command,
operable program or batch file.
'gzip' is not recognized as an internal or external command,
operable program or batch file.


In [240]:
import jsonlines

dataset_file = "github-typo-corpus.v1.0.0.jsonl"

dataset = []
other_langs = set()

with jsonlines.open(dataset_file) as reader:
    for obj in reader:
        for edit in obj['edits']:
            if edit['src']['lang'] == 'eng' and edit['is_typo']:
                src, tgt = edit['src']['text'], edit['tgt']['text']
                if src.lower() != tgt.lower():
                    dataset.append((src, tgt))

print(f"Dataset size = {len(dataset)}")


Dataset size = 245909


Please, explore the dataset. You may see, that this is

- mostly markdown
- some common mistakes with do/does
- some just refer to punctuation typos (which we do not consider)


In [241]:
for pair in dataset[1010:1020]:
    print(f"{pair[0]} => {pair[1]}")


        """Make am instance. =>         """Make an instance.
* travis: test agains Node.js 11 => * travis: test against Node.js 11
The parser receive a string and returns an array inside a user-provided  => The parser receives a string and returns an array inside a user-provided 
CSV data is send through the `write` function and the resulted data is obtained => CSV data is sent through the `write` function and the resulting data is obtained
One useful function part of the Stream API is `pipe` to interact between  => One useful function of the Stream API is `pipe` to interact between 
source to a `stream.Writable` object destination. This example available as  => source to a `stream.Writable` object destination. This example is available as 
`node samples/pipe.js` read the file, parse its content and transform it. => `node samples/pipe.js` and reads the file, parses its content and transforms it.
Most of the generator is imported from its parent project [CSV][csv] in a effort  => Most o

Compare your implementation with Norvig's solution on this dataset


In [246]:
limit = 10000
counter = limit
my_solution = MySolution('w2_.txt')
total_words = 0
my_correct = 0
norv_correct = 0
wrong = []
wrong2 = []
try:
    for i, (src, target) in enumerate(dataset):
        if i == limit:
            break
        # YOUR CODE HERE
        x = words(src)
        y = words(target)
        my_result = my_solution.check(x)
        norvig_result = correction(x)

        for j in range(len(x)):
            try:
                if my_result[j] == y[j]:
                    my_correct += 1
                else:
                    wrong.append((my_result[j], y[j]))
                if norvig_result[j] == y[j]:
                    norv_correct += 1
                else:
                    wrong2.append(norvig_result[j])
            except:
                pass
            total_words += 1
except:
    pass
print(f'My solution got {my_correct/total_words*100}%')
print(f'Norvig solution got {norv_correct/total_words*100}%')


My solution got 79.77807666442501%
Norvig solution got 78.0497646267653%
